In [255]:
import networkx as nx
from pathlib import Path
import pandas as pd
import numpy as np
import community as c

In [256]:
data_dir = Path(".").absolute().parent / "data"
fig_dir = Path(".").absolute().parent / "figs"

In [403]:
df = (
    pd.read_csv(data_dir / "rsctest_LC_sizes.csv")
    .loc[lambda x: x.binsize == 0.2]
    .assign(r= lambda x: x.R_spike_count)
    .loc[:, ["spiketrain_1", "spiketrain_2", "r", "p", "spiketrain_1_cluster", "spiketrain_2_cluster", "session_name"]]
)

In [359]:
df = (
    pd.read_csv(data_dir / "evoked_rsc.csv")
    .loc[:, ["spiketrain_1", "spiketrain_2", "r", "p", "session_name"]]
)

In [395]:
def create_graph(
    df: pd.core.frame.DataFrame, 
    st1_col: str = "spiketrain_1",
    st2_col: str = "spiketrain_2",
    corr_col: str = "r",
    corr_rank_method: str = "mag",
    degree_threshold_method: str = "log_n_nodes"
):
    """
    Construct a graph for a single recording session.
    
    A node is created for each neuron. An edge is drawn between neurons
    with high correlation until the mean network degree reaches a threshold value.
    
    Args:
        df: A pandas dataframe containing one row for each correlated neuron pair.
            Must only contain data from a single recording session.
        st1_col: The name of the column in df containing the first neuron in the correlation
        st2_col: The name of the column in df containing the second neuron in the correlation
        corr_col: The name of the column in df containing the correlation coeficient
        corr_rank_method: Method to use when ranking correlations {"mag", "identity"}. 
                          'mag'      - ranks correlations by their magnitude,
                          'identity' - ranks correlations by their real value.
                          'min'      - lowest to highest
        degree_threshold_method: Method to use when calculating the threshold for mean degree
                                 centrality. {"log_n_nodes"}
                                 'log_n_nodes' - log10 of the number of nodes in the network
                                 'all' - Connect each correlation
    Returns:
        A networkx undirected Graph object
    """
    # get set of nodes -> neuron_ids
    st1, st2 = df[st1_col].unique(), df[st2_col].unique()
    neuron_ids = np.union1d(df.spiketrain_1.unique(), df.spiketrain_1.unique())
    
    # construct an empty graph and initialise the nodes
    G = nx.Graph()
    G.add_nodes_from(neuron_ids)
    
    # parse correlation ranking method
    if corr_rank_method == "mag":
        df = df.assign(corr_mag=lambda x: np.abs(x[corr_col]))
        corr_col = "corr_mag"
    elif corr_rank_method == "identity":
        pass
    else:
        raise ValueError(f"Unknown Correlation Ranking Method: {corr_rank_method}")
    
    # parse degree threshold
    if degree_threshold_method == "log_n_nodes":
        thresh = np.log(len(nx.nodes(G)))
    elif degree_threshold_method == "all":
        pass
    else:
        raise ValueError(f"Unknown Degree Threshold: {degree_threshold_method}")
    
    # draw edges between nodes until threshold has been met
    for row in df.sort_values(corr_col).to_dict(orient="row"):
        n1, n2 = row[st1_col], row[st2_col]
        
        G.add_edge(n1, n2)
        
        if degree_threshold_method != "all":
            mean_degree = np.mean(list(dict(G.degree).values()))
            if mean_degree >= thresh:
                G.remove_edge(n1, n2)
                break
                
    return G

In [396]:
def contruct_surrogate_graphs(
    df: pd.core.frame.DataFrame,
    n:int = 100,
    st1_col: str = "spiketrain_1",
    st2_col: str = "spiketrain_2",
    corr_col: str = "r",
    corr_rank_method: str = "mag",
    degree_threshold_method: str = "log_n_nodes"
):
    """
    Construct surrogate graphs by permuting correlations.
    
    Args:
        df: A pandas dataframe containing one row for each correlated neuron pair.
            Must only contain data from a single recording session.
        st1_col: The name of the column in df containing the first neuron in the correlation
        st2_col: The name of the column in df containing the second neuron in the correlation
        corr_col: The name of the column in df containing the correlation coeficient
        corr_rank_method: Method to use when ranking correlations {"mag", "identity"}. 
                          'mag'      - ranks correlations by their magnitude,
                          'identity' - ranks correlations by their real value.
        degree_threshold_method: Method to use when calculating the threshold for mean degree
                                 centrality. {"log_n_nodes"}
                                 'log_n_nodes' - log10 of the number of nodes in the network
    Returns:
        A networkx undirected Graph object
    """
    graphs = []
    for i in range(n):
        graphs.append(
            df.assign(st1_perm=lambda x: np.random.permutation(x[st1_col]),
                      st2_perm=lambda x: np.random.permutation(x[st2_col]))
            .pipe(lambda x: create_graph(
                            df=x, st1_col="st1_perm", st2_col="st2_perm", 
                            corr_col=corr_col, corr_rank_method=corr_rank_method,
                            degree_threshold_method=degree_threshold_method
                           )
                 )
        )
    return graphs

In [397]:
def test_modularity(G, n=500):
    n_nodes = len(G.nodes())
    n_edges = len(G.edges())
    s_graphs = [nx.generators.random_graphs.gnm_random_graph(
                    n=n_nodes, 
                    m=n_edges,
                ) for i in range(n)]
    mods = np.array(list(map(get_modularity, s_graphs)))
    mod = get_modularity(G)
    return np.mean(mods >= mod), mods.mean(), mod

In [398]:
def neuron_ids(df):
    return np.union1d(df.spiketrain_1.unique(), df.spiketrain_1.unique())

In [399]:
def get_modularity(G):
    partition = c.best_partition(G)
    return c.modularity(partition, G)

In [405]:
sessions = df["session_name"].unique()
d = []
for i, session in enumerate(sessions):
    print(f"{(i + 1)/len(sessions) * 100}%")
    df1 = (
        df
        .loc[lambda x: x.session_name == session]
    )
    G = create_graph(df1, degree_threshold_method="log_n_nodes")
    p, m, ob = test_modularity(G, n=1000)
    d.append({"session": session, "p": p, "mean": m, "obs": ob})

4.545454545454546%
5.545454545454546%
6.545454545454546%
7.545454545454546%
8.545454545454547%
9.545454545454547%
10.545454545454547%
11.545454545454547%
12.545454545454547%
13.545454545454547%
14.545454545454547%
15.545454545454547%
16.545454545454547%
17.545454545454547%
18.545454545454547%
19.545454545454547%
20.545454545454547%
21.545454545454547%
22.545454545454547%
23.545454545454547%
24.545454545454547%
25.545454545454547%


In [406]:
pd.DataFrame(d)

,session,p,mean,obs
0,ESHOCK_03_LOC1,1.000,0.194278,0.000000
1,ESHOCK_04_LOC1,0.987,0.392902,0.314694
2,ESHOCK_06_LOC1,0.963,0.389787,0.318939
3,ESHOCK_07_LOC1,0.998,0.330714,0.160714
4,ESHOCK_08_LOC1,0.889,0.409950,0.375857
5,ESHOCK_09_LOC1,1.000,0.415656,0.330688
6,hamilton_10,0.969,0.247403,0.111111
7,hamilton_03,0.845,0.295259,0.222222
8,hamilton_04,0.884,0.350518,0.293750
9,hamilton_09,0.996,0.336670,0.201172


In [389]:
G = nx.karate_club_graph()

In [390]:
test_modularity(G, n=500)

(0.0, 0.3443251150558843, 0.4188034188034188)